# Set up an image classifier to identify dogs vs muffins

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-07-28 13:48:13.099455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Lets load in our data set and mix em up a bit
seed_train_validation = 1 # Seed for repeatability

train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.2,
                                  shear_range=0.2,
                                  rotation_range=10,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

# Load our training dataset
train_ds = keras.utils.image_dataset_from_directory(
    directory ='/Users/tktakaro/Documents/ML_Projects/MuffinDog/train',
    image_size = (150,150),
    seed = seed_train_validation,
    color_mode='rgb',
    label_mode = 'categorical',
    batch_size=32,
    shuffle = True)

test_datagen = ImageDataGenerator(rescale=1./255)
# Load our test dataset
test_val_ds = keras.utils.image_dataset_from_directory(
    directory ='/Users/tktakaro/Documents/ML_Projects/MuffinDog/test',
    image_size = (150,150),
    seed = seed_train_validation,
    label_mode = 'categorical',
    batch_size=32,
    color_mode = 'rgb',
    shuffle = True)

Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.


In [3]:
# Split the test dataset in half to give us a validation dataset too
batches = tf.data.experimental.cardinality(test_val_ds) # How many files in test dataset?
test_ds = test_val_ds.take(batches // 2)
val_ds = test_val_ds.skip(batches // 2)

In [5]:
# First, use ResNet as a base model
base_model = keras.applications.ResNet50(weights='imagenet', include_top = False, input_shape=(150,150,3))
# Freeze layers from base_model to only train new specialized layers for this problem
for layer in base_model.layers:
    layer.trainable = False
        
# Build up layers
x = base_model.output    
x = (keras.layers.Dropout(0.2))(x)    
x = keras.layers.Flatten()(x)
# Two categories: Muffins and Dogs
predictions = keras.layers.Dense(2, activation = 'softmax')(x)

# Set up and compile model
model = keras.models.Model(inputs = base_model.input , outputs =  predictions)
    
model.compile(optimizer='Nadam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Run the fit
model.fit(train_ds, 
    epochs = 10 , validation_data=val_ds,
    steps_per_epoch = len(train_ds)//10,
    validation_steps=len(val_ds),)

Epoch 1/10
14/14 [==============================] - 34s 2s/step - loss: 0.9995 - accuracy: 0.8772 - val_loss: 0.2640 - val_accuracy: 0.9655
Epoch 2/10
14/14 [==============================] - 32s 2s/step - loss: 0.4265 - accuracy: 0.9621 - val_loss: 0.2631 - val_accuracy: 0.9720
Epoch 3/10
14/14 [==============================] - 33s 2s/step - loss: 0.2786 - accuracy: 0.9710 - val_loss: 0.4454 - val_accuracy: 0.9507
Epoch 4/10
14/14 [==============================] - 33s 2s/step - loss: 0.3093 - accuracy: 0.9621 - val_loss: 0.2293 - val_accuracy: 0.9737
Epoch 5/10
14/14 [==============================] - 32s 2s/step - loss: 0.4040 - accuracy: 0.9665 - val_loss: 0.2383 - val_accuracy: 0.9688
Epoch 6/10
14/14 [==============================] - 32s 2s/step - loss: 0.1468 - accuracy: 0.9799 - val_loss: 0.2073 - val_accuracy: 0.9720
Epoch 7/10
14/14 [==============================] - 32s 2s/step - loss: 0.1982 - accuracy: 0.9754 - val_loss: 0.2611 - val_accuracy: 0.9737
Epoch 8/10
14/14 [==

## Now lets try messing with a tuner to find the best optimizer

In [6]:
import keras_tuner
from keras_tuner import RandomSearch

def built_finetune_model(hp): 
    # Same model architecture as before, just now adding an optimizer chooser
    base_model = keras.applications.ResNet50(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3),)
        
    for layer in base_model.layers:
        layer.trainable = False
        
    x = base_model.output
    x = (keras.layers.Dropout(0.2))(x)
    x = keras.layers.Flatten()(x)

    predictions = keras.layers.Dense(2 , activation = 'softmax')(x)

    finetune_model = keras.models.Model(inputs = base_model.input , outputs =  predictions)
    # Set up optimizers
    adam = keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4]))
    sgd = keras.optimizers.SGD(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4]))
    rmsprop = keras.optimizers.RMSprop(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4]))
    nadam = keras.optimizers.Nadam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4]))
    adadelta = keras.optimizers.Adadelta(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4]))
    
    # Choose and compile
    optimizer1 = hp.Choice('optimizer' , values = ['adam','sgd','rmsprop','nadam','adadelta'])
    finetune_model.compile(optimizer =optimizer1,
                           loss = 'categorical_crossentropy',
                           metrics = ['accuracy'])
    return finetune_model

In [7]:
# Set up tuner to search for best model
tuner = RandomSearch(
    built_finetune_model,
    keras_tuner.Objective("accuracy", direction="max"),
    max_trials = 5,
    executions_per_trial = 1,
    directory = 'my_dir_resnet1',

)
# Run tuner
tuner.search(
    train_ds , 
    epochs = 2 , 
    validation_data=test_ds,
    steps_per_epoch = len(train_ds)/10 ,
    validation_steps=len(test_ds),
)

Trial 5 Complete [00h 01m 13s]
accuracy: 0.9770833253860474

Best accuracy So Far: 0.9770833253860474
Total elapsed time: 00h 03m 33s
INFO:tensorflow:Oracle triggered exit


In [8]:
# Extract params from best model
tuner.get_best_hyperparameters()[0].values
model = tuner.get_best_models(num_models =1)[0]
# Run fit to fine tune
model.fit(train_ds , 
    epochs = 10 , validation_data=val_ds,
    steps_per_epoch = len(train_ds) ,
    validation_steps=len(val_ds),)

Epoch 1/10
148/148 [==============================] - 168s 1s/step - loss: 0.8673 - accuracy: 0.9755 - val_loss: 0.9217 - val_accuracy: 0.9720
Epoch 2/10
148/148 [==============================] - 168s 1s/step - loss: 0.2460 - accuracy: 0.9856 - val_loss: 0.4435 - val_accuracy: 0.9836
Epoch 3/10
148/148 [==============================] - 169s 1s/step - loss: 0.0764 - accuracy: 0.9962 - val_loss: 0.5643 - val_accuracy: 0.9819
Epoch 4/10
148/148 [==============================] - 177s 1s/step - loss: 0.0338 - accuracy: 0.9975 - val_loss: 0.6450 - val_accuracy: 0.9803
Epoch 5/10
148/148 [==============================] - 182s 1s/step - loss: 0.0147 - accuracy: 0.9983 - val_loss: 0.6280 - val_accuracy: 0.9836
Epoch 6/10
148/148 [==============================] - 174s 1s/step - loss: 0.0134 - accuracy: 0.9985 - val_loss: 0.7107 - val_accuracy: 0.9836
Epoch 7/10
148/148 [==============================] - 175s 1s/step - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.4884 - val_accuracy: 0.9819

In [9]:
# Evaluate the accuracy using the test dataset
score = model.evaluate(test_ds, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.3389919102191925
Test accuracy: 0.9913194179534912
